## Imports

In [150]:
## Imports
import os
import re
import numpy as np
import pandas as pd

from sklearn.metrics import precision_recall_fscore_support, accuracy_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression

import gensim.downloader
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
)

## Tokenizer

In [151]:
word_tokenize_pattern = re.compile(r"(?u)\b\w\w+\b")


def word_tokenize(s: str):
    return [x.lower() for x in word_tokenize_pattern.findall(s)]

## Metrics

In [152]:

EMOTIONS = ["Angry", "Happy", "Relaxed", "Sad"]
metrics_result = []
emotions_metric_result = []





def print_results(gold_labels, predicted_labels, model_name = "", display = True):
    # overall
    p, r, f, _ = precision_recall_fscore_support(
        gold_labels, predicted_labels, average="macro", zero_division=0
    )
    acc = accuracy_score(gold_labels, predicted_labels)

    result = {"Model": model_name, "Precision": p, "Recall": r, "F1": f, "Accuracy": acc}
    

    index_found = next((i for i, d in enumerate(metrics_result) if d.get('Model') == model_name), None)

    if (index_found == None):
        metrics_result.append(result)
    else:
        metrics_result[index_found] = result

    df_all = pd.DataFrame(metrics_result)
  
    if (display == True):
        print("=== Overall (Macro Avg) ===")
        # print("Precision:", p)
        # print("Recall:", r)
        # print("F1:", f)
        # print("Accuracy:", acc)
        # print(metrics_result)
        print(df_all.to_string(index=False))
        print()

   


    # Per-emotion metrics
    p_i, r_i, f_i, _ = precision_recall_fscore_support(
        gold_labels, predicted_labels, average=None, zero_division=0
    )
    if (display == True):
        print("=== Per Emotion (Class) Metrics ===")
    for i, emotion in enumerate(EMOTIONS):
        # print("  Precision:", p_i[i])
        # print("  Recall:   ", r_i[i])
        # print("  F1:       ", f_i[i])
        emotion_result = {"Model": model_name, "Emotion": emotion, "Precision": p_i[i], "Recall": r_i[i], "F1": f_i[i]}
        emotion_index_found = next((i for i, d in enumerate(emotions_metric_result) if d.get('Model') == model_name and d.get("Emotion" == emotion)), None)

        if (emotion_index_found == None):
            emotions_metric_result.append(emotion_result)
        else:
            emotions_metric_result[emotion_index_found] = emotion_result
        df_emotions = pd.DataFrame(emotions_metric_result)
        filtered_df = df_emotions[df_emotions['Emotion'] == emotion]
        if (display == True):
            print(f"{emotion}:")
            print(filtered_df.to_string(index=False))
            print()
    
    
    
   


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = logits.argmax(axis=-1)

    p, r, f, _ = precision_recall_fscore_support(
        labels, preds, average="macro", zero_division=0
    )
    acc = accuracy_score(labels, preds)
    return {"precision": p, "recall": r, "f1": f, "accuracy": acc}




## Dataset

In [153]:
DATASET_DIR = "NJU_MusicMood_v1.0"

label2id = {label: i for i, label in enumerate(EMOTIONS)}
id2label = {i: label for label, i in label2id.items()}

timestamp_pattern = re.compile(r"\[\d{2}:\d{2}(?:\.\d{2})?\]")


def clean_lyrics(text: str) -> str:
    # Remove timestamps like [00:29]
    text = timestamp_pattern.sub("", text)

    # Lowercase
    text = text.lower()

    # Normalize quotes
    text = text.replace("’", "'").replace("“", '"').replace("”", '"')

    # Remove ellipses and repeated dots
    text = re.sub(r"\.{2,}", " ", text)

    # Remove long underscores
    text = re.sub(r"_{2,}", " ", text)

    # Remove trailing "end" markers at the end of the file
    text = re.sub(r"\bend[.\s]*$", "", text.strip())

    # Replace newlines with space
    text = text.replace("\n", " ")

    # Keep only letters, digits, spaces, apostrophes
    text = re.sub(r"[^a-z0-9' ]+", " ", text)

    # Collapse multiple spaces
    text = re.sub(r"\s+", " ", text)

    return text.strip()



def get_lyrics(path: str) -> str:
    with open(path, "r", encoding="utf-8") as f:
        raw = f.read()
    return clean_lyrics(raw)


def get_lyrics_and_labels(split: str):
    texts, labels = [], []
    for emotion in EMOTIONS:
        folder = os.path.join(DATASET_DIR, emotion, split)
        if not os.path.isdir(folder):
            continue

        for fname in os.listdir(folder):
            if not fname.endswith(".txt"):
                continue
            if fname.lower() == "info.txt":
                continue

            path = os.path.join(folder, fname)
            txt = get_lyrics(path)
            if txt.strip():
                texts.append(txt)
                labels.append(emotion)
    return texts, labels


# Load data 
train_texts, train_labels = get_lyrics_and_labels("Train")
dev_texts, dev_labels = get_lyrics_and_labels("Test")

assert len(train_texts) == len(train_labels)
assert len(dev_texts) == len(dev_labels)

# Datasets
train_ds = Dataset.from_dict(
    {"text": train_texts, "label": [label2id[l] for l in train_labels]}
)
dev_ds = Dataset.from_dict(
    {"text": dev_texts, "label": [label2id[l] for l in dev_labels]}
)


## Baseline: Bag of Words & Logistic Regression

In [154]:
print("=== Baseline: Bag of Words & Logistic Regression ===")

count_vectorizer = CountVectorizer(analyzer=word_tokenize)
train_counts = count_vectorizer.fit_transform(train_texts)
dev_counts = count_vectorizer.transform(dev_texts)

lr_bow = LogisticRegression(max_iter=500, random_state=0)
lr_bow.fit(train_counts, train_labels)

lr_bow_dev_predictions = lr_bow.predict(dev_counts)
print_results(dev_labels, lr_bow_dev_predictions, "BoW & LR")

=== Baseline: Bag of Words & Logistic Regression ===
=== Overall (Macro Avg) ===
   Model  Precision   Recall       F1  Accuracy
BoW & LR   0.380262 0.372645 0.373144  0.363395

=== Per Emotion (Class) Metrics ===
Angry:
   Model Emotion  Precision   Recall       F1
BoW & LR   Angry   0.492958 0.492958 0.492958

Happy:
   Model Emotion  Precision   Recall       F1
BoW & LR   Happy    0.45679 0.349057 0.395722

Relaxed:
   Model Emotion  Precision  Recall       F1
BoW & LR Relaxed   0.305344 0.39604 0.344828

Sad:
   Model Emotion  Precision   Recall       F1
BoW & LR     Sad   0.265957 0.252525 0.259067



## Baseline 2: Word2Vec & Logistic Regression

In [155]:
print("=== Word2Vec & Logistic Regression ===")

w2v_model = gensim.downloader.load("word2vec-google-news-300")
VECTOR_SIZE = w2v_model.vector_size


def vec_for_doc(tokenized_doc):
    vectors = [w2v_model[word] for word in tokenized_doc if word in w2v_model.key_to_index]
    if not vectors:
        return np.zeros(VECTOR_SIZE, dtype="float32")
    return np.mean(vectors, axis=0)


train_vecs = [vec_for_doc(word_tokenize(x)) for x in train_texts]
dev_vecs = [vec_for_doc(word_tokenize(x)) for x in dev_texts]

lr_w2v = LogisticRegression(max_iter=500, random_state=0)
lr_w2v.fit(train_vecs, train_labels)

w2v_dev_predictions = lr_w2v.predict(dev_vecs)
print_results(dev_labels, w2v_dev_predictions, "Word2Vec & LR")

=== Word2Vec & Logistic Regression ===
=== Overall (Macro Avg) ===
        Model  Precision   Recall       F1  Accuracy
     BoW & LR   0.380262 0.372645 0.373144  0.363395
Word2Vec & LR   0.461446 0.478161 0.455341  0.453581

=== Per Emotion (Class) Metrics ===
Angry:
        Model Emotion  Precision   Recall       F1
     BoW & LR   Angry   0.492958 0.492958 0.492958
Word2Vec & LR   Angry   0.612903 0.802817 0.695122

Happy:
        Model Emotion  Precision   Recall       F1
     BoW & LR   Happy   0.456790 0.349057 0.395722
Word2Vec & LR   Happy   0.480392 0.462264 0.471154

Relaxed:
        Model Emotion  Precision   Recall       F1
     BoW & LR Relaxed   0.305344 0.396040 0.344828
Word2Vec & LR Relaxed   0.335821 0.445545 0.382979

Sad:
        Model Emotion  Precision   Recall       F1
     BoW & LR     Sad   0.265957 0.252525 0.259067
Word2Vec & LR     Sad   0.416667 0.202020 0.272109



## Split lyrics into start middle end

In [183]:
def get_segment(text, segment="start", portion=0.33):
    """
    Extract a portion of the lyrics.
    portion=0.3 means 30% of tokens.
    segment can be "start", "middle", or "end".
    """
    tokens = text.split()
    n = len(tokens)
    if n == 0:
        return ""

    cut = int(n * portion)  # number of tokens for start/end

    if segment == "start":
        return " ".join(tokens[:cut])

    elif segment == "middle":
        start = int(n * 0.33)
        end = int(n * 0.66)
        return " ".join(tokens[start:end])

    elif segment == "end":
        laststart = int(n * 0.66)
        # lastend= int(n * (portion + 0.01))
        # return " ".join(tokens[-lastcut:])
        return " ".join(tokens[laststart:n])

    else:
        return text


# Build dev
dev_start_texts = [get_segment(t, "start") for t in dev_texts]
dev_middle_texts = [get_segment(t, "middle") for t in dev_texts]
dev_end_texts = [get_segment(t, "end") for t in dev_texts]



## Evaluate BoW model Segmented Lyrics

In [157]:
print("=== BoW Logistic Regression: Position-based Evaluation (NO retraining) ===")

# START
dev_start_counts = count_vectorizer.transform(dev_start_texts,)
bow_start_preds = lr_bow.predict(dev_start_counts)
# print("\n--- BoW on START segment only ---")
print_results(dev_labels, bow_start_preds, "BoW Segmented_START", False)

# MIDDLE
dev_middle_counts = count_vectorizer.transform(dev_middle_texts)
bow_middle_preds = lr_bow.predict(dev_middle_counts)
# print("\n--- BoW on MIDDLE segment only ---")
print_results(dev_labels, bow_middle_preds, "BoW Segmented_MIDDLE", False)

# END
dev_end_counts = count_vectorizer.transform(dev_end_texts)
bow_end_preds = lr_bow.predict(dev_end_counts)
# print("\n--- BoW on END segment only ---")
print_results(dev_labels, bow_end_preds, "BoW Segmented_END")


=== BoW Logistic Regression: Position-based Evaluation (NO retraining) ===
=== Overall (Macro Avg) ===
               Model  Precision   Recall       F1  Accuracy
            BoW & LR   0.380262 0.372645 0.373144  0.363395
       Word2Vec & LR   0.461446 0.478161 0.455341  0.453581
 BoW Segmented_START   0.414294 0.320851 0.275773  0.323607
BoW Segmented_MIDDLE   0.405777 0.314621 0.274071  0.318302
   BoW Segmented_END   0.341373 0.301432 0.271661  0.307692

=== Per Emotion (Class) Metrics ===
Angry:
               Model Emotion  Precision   Recall       F1
            BoW & LR   Angry   0.492958 0.492958 0.492958
       Word2Vec & LR   Angry   0.612903 0.802817 0.695122
 BoW Segmented_START   Angry   0.575758 0.267606 0.365385
BoW Segmented_MIDDLE   Angry   0.562500 0.253521 0.349515
   BoW Segmented_END   Angry   0.441176 0.211268 0.285714

Happy:
               Model Emotion  Precision   Recall       F1
            BoW & LR   Happy   0.456790 0.349057 0.395722
       Word2Vec & LR 

## Evaluate Word2Vec on Segmented Lyrics

In [158]:
print("=== Word2Vec Logistic Regression: Position-based Evaluation (NO retraining) ===")

# START
dev_start_vecs = [vec_for_doc(word_tokenize(x)) for x in dev_start_texts]
w2v_start_preds = lr_w2v.predict(dev_start_vecs)
# print("\n--- W2V on START segment only ---")
print_results(dev_labels, w2v_start_preds, "Word2Vec Segmented_START", False)

# MIDDLE
dev_middle_vecs = [vec_for_doc(word_tokenize(x)) for x in dev_middle_texts]
w2v_middle_preds = lr_w2v.predict(dev_middle_vecs)
# print("\n--- W2V on MIDDLE segment only ---")
print_results(dev_labels, w2v_middle_preds, "Word2Vec Segmented_MIDDLE", False)

# END
dev_end_vecs = [vec_for_doc(word_tokenize(x)) for x in dev_end_texts]
w2v_end_preds = lr_w2v.predict(dev_end_vecs)
# print("\n--- W2V on END segment only ---")
print_results(dev_labels, w2v_end_preds, "Word2Vec Segmented_END")

=== Word2Vec Logistic Regression: Position-based Evaluation (NO retraining) ===
=== Overall (Macro Avg) ===
                    Model  Precision   Recall       F1  Accuracy
                 BoW & LR   0.380262 0.372645 0.373144  0.363395
            Word2Vec & LR   0.461446 0.478161 0.455341  0.453581
      BoW Segmented_START   0.414294 0.320851 0.275773  0.323607
     BoW Segmented_MIDDLE   0.405777 0.314621 0.274071  0.318302
        BoW Segmented_END   0.341373 0.301432 0.271661  0.307692
 Word2Vec Segmented_START   0.409452 0.430841 0.409294  0.411141
Word2Vec Segmented_MIDDLE   0.427968 0.456468 0.428019  0.432361
   Word2Vec Segmented_END   0.388004 0.411518 0.388873  0.389920

=== Per Emotion (Class) Metrics ===
Angry:
                    Model Emotion  Precision   Recall       F1
                 BoW & LR   Angry   0.492958 0.492958 0.492958
            Word2Vec & LR   Angry   0.612903 0.802817 0.695122
      BoW Segmented_START   Angry   0.575758 0.267606 0.365385
     BoW Se

## Helpers

In [159]:
def tokenize_dataset(dataset, tokenizer, max_length: int = 256):
    def _tok(batch):
        return tokenizer(
            batch["text"],
            truncation=True,
            padding="max_length",
            max_length=max_length,
        )

    tokenized = dataset.map(_tok, batched=True)
    tokenized = tokenized.remove_columns(["text"])
    tokenized.set_format(type="torch")
    return tokenized


def train_and_eval_transformer(
    model_name: str,
    train_dataset: Dataset,
    dev_dataset: Dataset,
    output_dir: str,
    num_epochs: int,
    learning_rate: float,
    train_bs: int,
    eval_bs: int,
    model_directory: str,
    set_pad_token_eos: bool = False,
):
    print(f"=== Fine-tuning {model_name} ===")

    tokenizer = AutoTokenizer.from_pretrained(model_name)

    if set_pad_token_eos:
        tokenizer.pad_token = tokenizer.eos_token

    tokenized_train = tokenize_dataset(train_dataset, tokenizer)
    tokenized_dev = tokenize_dataset(dev_dataset, tokenizer)

    model = AutoModelForSequenceClassification.from_pretrained(
        model_name,
        num_labels=len(EMOTIONS),
        label2id=label2id,
        id2label=id2label,
        ignore_mismatched_sizes=True,
    )

    if set_pad_token_eos:
        model.config.pad_token_id = tokenizer.eos_token_id

    training_args = TrainingArguments(
        output_dir=output_dir,
        num_train_epochs=num_epochs,
        learning_rate=learning_rate,
        per_device_train_batch_size=train_bs,
        per_device_eval_batch_size=eval_bs,
        weight_decay=0.01,
        eval_strategy="epoch",
        logging_steps=16,
        log_level="error",
        report_to="none",
        save_strategy="epoch",
        dataloader_pin_memory=False,
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_train,
        eval_dataset=tokenized_dev,
        compute_metrics=compute_metrics,
        tokenizer=tokenizer,
    )

    trainer.train()

    eval_results = trainer.evaluate()
    print(f"{model_name} dev results:", eval_results)

    pred_output = trainer.predict(tokenized_dev)
    logits = pred_output.predictions
    pred_ids = np.argmax(logits, axis=-1)
    pred_labels = [id2label[i] for i in pred_ids]

    print(f"{model_name} classification report:")
    print_results(dev_labels, pred_labels, model_name)

    trainer.save_model(model_directory)
    tokenizer.save_pretrained(model_directory)

    return trainer, eval_results, pred_labels

def eval_transformer_on_segments(model_name: str, trainer: Trainer):
    """
    Evaluate a fine-tuned transformer (trainer) on START/MIDDLE/END
    segments of the dev set, without retraining.
    """
    tokenizer = trainer.tokenizer

    for seg in ["start", "middle", "end"]:
        # Build segmented dev texts
        seg_texts = [get_segment(t, seg) for t in dev_texts]

        # Build a segmented dev Dataset with same labels
        seg_dev_ds = Dataset.from_dict(
            {
                "text": seg_texts,
                "label": [label2id[l] for l in dev_labels],
            }
        )

        # Tokenize segmented dataset
        tokenized_seg_dev = tokenize_dataset(seg_dev_ds, tokenizer)

        # Predict
        pred_output = trainer.predict(tokenized_seg_dev)
        logits = pred_output.predictions
        pred_ids = np.argmax(logits, axis=-1)
        pred_labels = [id2label[i] for i in pred_ids]

        print(f"\n=== {model_name} on {seg.upper()} segment only ===")
        # print_results(dev_labels, pred_labels, model_name, False)
        if (seg == "end"):
            print_results(dev_labels, pred_labels, f"{model_name}_{seg}")
        else:
            print_results(dev_labels, pred_labels, f"{model_name}_{seg}", False)


## Distilbert

In [160]:
import shutil
model_name = "distilbert/distilbert-base-uncased"
distilbert_model_directory = "distilbert_model"
if os.path.isdir(model_name):
    print(f"Trained Model existing in directory '{distilbert_model_directory}'")
    try:
        # os.rmdir(distilbert_model_directory)
        shutil.rmtree(distilbert_model_directory)
        print(f"The saved model '{distilbert_model_directory}' deleted successfully.")
    except OSError as e:
        print(f"Error deleting folder '{distilbert_model_directory}': {e}")
else:
    print(f"The directory '{distilbert_model_directory}' does not exist.")

distilbert_trainer, distilbert_results, distilbert_pred_labels = train_and_eval_transformer(
    model_name=model_name,
    train_dataset=train_ds,
    dev_dataset=dev_ds,
    output_dir="./distilbert_musicmood",
    num_epochs=3,
    learning_rate=5e-5,
    train_bs=4,
    eval_bs=4,
    model_directory=distilbert_model_directory,
    set_pad_token_eos=False,
)

Trained Model existing in directory 'distilbert_model'
The saved model 'distilbert_model' deleted successfully.
=== Fine-tuning distilbert/distilbert-base-uncased ===


Map:   0%|          | 0/400 [00:00<?, ? examples/s]

Map:   0%|          | 0/377 [00:00<?, ? examples/s]

C:\Users\HP\AppData\Local\Temp\ipykernel_45776\835601709.py:64: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


{'loss': 0.5069, 'grad_norm': 5.180815696716309, 'learning_rate': 4.75e-05, 'epoch': 0.16}
{'loss': 0.83, 'grad_norm': 7.2908711433410645, 'learning_rate': 4.483333333333333e-05, 'epoch': 0.32}
{'loss': 0.7159, 'grad_norm': 4.195237159729004, 'learning_rate': 4.216666666666667e-05, 'epoch': 0.48}
{'loss': 0.5563, 'grad_norm': 21.9300479888916, 'learning_rate': 3.9500000000000005e-05, 'epoch': 0.64}
{'loss': 0.5397, 'grad_norm': 29.086624145507812, 'learning_rate': 3.683333333333334e-05, 'epoch': 0.8}
{'loss': 0.7083, 'grad_norm': 9.722677230834961, 'learning_rate': 3.4166666666666666e-05, 'epoch': 0.96}
{'eval_loss': 1.7001217603683472, 'eval_precision': 0.4511138463131101, 'eval_recall': 0.4570951547719215, 'eval_f1': 0.42132056848901406, 'eval_accuracy': 0.4297082228116711, 'eval_runtime': 62.6141, 'eval_samples_per_second': 6.021, 'eval_steps_per_second': 1.517, 'epoch': 1.0}
{'loss': 0.7769, 'grad_norm': 18.317607879638672, 'learning_rate': 3.15e-05, 'epoch': 1.12}
{'loss': 0.4088,

## Segment Test

In [161]:
eval_transformer_on_segments("DistilBERT", distilbert_trainer)

Map:   0%|          | 0/377 [00:00<?, ? examples/s]


=== DistilBERT on START segment only ===


Map:   0%|          | 0/377 [00:00<?, ? examples/s]


=== DistilBERT on MIDDLE segment only ===


Map:   0%|          | 0/377 [00:00<?, ? examples/s]


=== DistilBERT on END segment only ===
=== Overall (Macro Avg) ===
                             Model  Precision   Recall       F1  Accuracy
                          BoW & LR   0.380262 0.372645 0.373144  0.363395
                     Word2Vec & LR   0.461446 0.478161 0.455341  0.453581
               BoW Segmented_START   0.414294 0.320851 0.275773  0.323607
              BoW Segmented_MIDDLE   0.405777 0.314621 0.274071  0.318302
                 BoW Segmented_END   0.341373 0.301432 0.271661  0.307692
          Word2Vec Segmented_START   0.409452 0.430841 0.409294  0.411141
         Word2Vec Segmented_MIDDLE   0.427968 0.456468 0.428019  0.432361
            Word2Vec Segmented_END   0.388004 0.411518 0.388873  0.389920
distilbert/distilbert-base-uncased   0.484091 0.489880 0.480128  0.466844
                  DistilBERT_start   0.458753 0.437947 0.438667  0.424403
                 DistilBERT_middle   0.464618 0.439572 0.443011  0.424403
                    DistilBERT_end   0.48861

## Test Model

In [199]:
from transformers import pipeline

try:

    # Load the model
    loaded_model = AutoModelForSequenceClassification.from_pretrained(distilbert_model_directory)

    # Load the tokenizer
    loaded_tokenizer = AutoTokenizer.from_pretrained(distilbert_model_directory)
    print("Model and tokenizer loaded successfully.")
except Exception as e:
    print(f"Error loading model: {e}")

# Create a pipeline using the loaded model and tokenizer
nlp = pipeline("text-classification", model=loaded_model, tokenizer=loaded_tokenizer)

# Example predictioni
lyrics_to_classify = """Every time we lie awake
After every hit we take
Every feeling that I get
But I haven't missed you yet

Every roommate kept awake
By every sigh and scream we make
All the feelings that I get
But I still don't miss you yet

Only when I stop to think about it

I hate everything about you
Why do I love you?
I hate everything about you
Why do I love you?

Every time we lie awake
After every hit we take
Every feeling that I get
But I haven't missed you yet

Only when I stop to think about it

I hate everything about you
Why do I love you?
I hate everything about you
Why do I love you?

Only when I stop to think about you
I know
Only when you stop to think about me
Do you know?

I hate everything about you
Why do I love you?
You hate everything about me
Why do you love me?

I hate
You hate
I hate
You love me

I hate everything about you
Why do I love you
"""

prediction = nlp(lyrics_to_classify)
print(prediction)

# Segment Prediction
for seg in ["start", "middle", "end"]:
    seg_text = get_segment(lyrics_to_classify, seg)
    print(f"\n=== Prediction on {seg.upper()} segment ===")
    # print(seg_text)
    seg_prediction = nlp(seg_text)
    print(seg_prediction)
    print()



Model and tokenizer loaded successfully.
[{'label': 'Angry', 'score': 0.9966964721679688}]

=== Prediction on START segment ===
[{'label': 'Angry', 'score': 0.9889976382255554}]


=== Prediction on MIDDLE segment ===
[{'label': 'Relaxed', 'score': 0.9770669341087341}]


=== Prediction on END segment ===
[{'label': 'Angry', 'score': 0.9876084327697754}]



## Sample Lyrics
**Ed Sheeran - Perfect**

"""I found a love for me
Darling just dive right in
And follow my lead
Well I found a girl beautiful and sweet 
I never knew you were the someone waiting for me
'Cause we were just kids when we fell in love

Not knowing what it was
I will not give you up this time
But darling, just kiss me slow, your heart is all I own
And in your eyes you're holding mine

Baby, I'm dancing in the dark with you between my arms
Barefoot on the grass, listening to our favorite song
When you said you alooked  mess, I whispered underneath my breath
But you heard it, darling, you look perfect tonight

Well I found a woman, stronger than anyone I know
She shares my dreams, I hope that someday I'll share her home
I found a love, to carry more than just my secrets
To carry love, to carry children of our own
We are still kids, but we're so in love
Fighting against all odds
I know we'll be alright this time
Darling, just hold my hand
Be my girl, I'll be your man
I see my future in your eyes

Baby, I'm dancing in the dark, with you between my arms
Barefoot on the grass, listening to our favorite song
When I saw you in that dress, looking so beautiful
I don't deserve this, darling, you look perfect tonight

Baby, I'm dancing in the dark, with you between my arms
Barefoot on the grass, listening to our favorite song
I have faith in what I see
Now I know I have met an angel in person
And she looks perfect
I don't deserve this
You look perfect tonight"""


**Pharrell Williams - Happy**


"""It might seem crazy what I'm about to say
Sunshine she's here, you can take a break
I'm a hot air balloon that could go to space
With the air, like I don't care, baby, by the way

Uh

(Because I'm happy)
Clap along if you feel like a room without a roof
(Because I'm happy)
Clap along if you feel like happiness is the truth
(Because I'm happy)
Clap along if you know what happiness is to you
(Because I'm happy)
Clap along if you feel like that's what you wanna do

Here come bad news, talking this and that (Yeah!)
Well, give me all you got, don't hold it back (Yeah!)
Well, I should probably warn you I'll be just fine (Yeah!)
No offense to you
Don't waste your time, here's why

(Because I'm happy)
Clap along if you feel like a room without a roof
(Because I'm happy)
Clap along if you feel like happiness is the truth
(Because I'm happy)
Clap along if you know what happiness is to you
(Because I'm happy)
Clap along if you feel like that's what you wanna do

Bring me down
Can't nothing bring me down
My level's too high to bring me down
Can't nothing bring me down, I said


(Because I'm happy)
Clap along if you feel like a room without a roof
(Because I'm happy)
Clap along if you feel like happiness is the truth
(Because I'm happy)
Clap along if you know what happiness is to you
(Because I'm happy)
Clap along if you feel like that's what you wanna do


Bring me down
Can't nothing bring me down
My level's too high to bring me down
Can't nothing bring me down, I said

(Because I'm happy)
Clap along if you feel like a room without a roof
(Because I'm happy)
Clap along if you feel like happiness is the truth
(Because I'm happy)
Clap along if you know what happiness is to you
(Because I'm happy)
Clap along if you feel like that's what you wanna do
(Because I'm happy)"""


**Katrina And The Waves Lyrics - I am Walking on Sunshine**


"""I used to think maybe you loved me, now, baby, I'm sure
And I just can't wait till the day when you knock on my door
Now every time I go for the mailbox, gotta hold myself down
'Cause I just can't wait till you write me you're coming around

I'm walking on sunshine, whoa
I'm walking on sunshine, whoa
I'm walking on sunshine, whoa
And don't it feel good
Hey, all right now
And don't it feel good
Hey, yeah

I used to think maybe you loved me, now I know that it's true
And I don't want to spend my whole life just a-waiting for you
Now, I don't want you back for the weekend, not back for a day, no, no, no
I said, baby, I just want you back, and I want you to stay

Oh, yeah, now I'm walking on sunshine, whoa
I'm walking on sunshine, whoa
I'm walking on sunshine, whoa
And don't it feel good
Hey, all right now
And don't it feel good
Yeah, oh, yeah, now
And don't it feel good

Walking on sunshine
Walking on sunshine

I feel alive, I feel the love, I feel the love that's really real
I feel alive, I feel the love, I feel the love that's really real
I'm on sunshine, baby, oh
Oh, yeah, I'm on sunshine, baby

Oh, I'm walking on sunshine, whoa
I'm walking on sunshine, whoa
I'm walking on sunshine, whoa
And don't it feel good
Hey, all right now
And don't it feel good
I'll say it, I'll say it, I'll say it again now
And don't it feel good
Hey, yeah now
And don't it feel good
Now don't it, don't it, don't it, don't it, don't it, don't it
And don't it feel good"""


**Adele - Someone Like you**

"""I heard that you're settled down
That you found a girl and you're married now.
I heard that your dreams came true.
Guess she gave you things I didn't give to you.

Old friend, why are you so shy?
Ain't like you to hold back or hide from the light.

I hate to turn up out of the blue uninvited
But I couldn't stay away, I couldn't fight it.
I had hoped you'd see my face and that you'd be reminded
That for me it isn't over.

Never mind, I'll find someone like you
I wish nothing but the best for you too
Don't forget me, I beg
I'll remember you said,
"Sometimes it lasts in love but sometimes it hurts instead,
Sometimes it lasts in love but sometimes it hurts instead"

You know how the time flies
Only yesterday was the time of our lives
We were born and raised
In a summer haze
Bound by the surprise of our glory days

I hate to turn up out of the blue uninvited
But I couldn't stay away, I couldn't fight it.
I'd hoped you'd see my face and that you'd be reminded
That for me it isn't over.

Never mind, I'll find someone like you
I wish nothing but the best for you too
Don't forget me, I beg
I'll remember you said,
"Sometimes it lasts in love but sometimes it hurts instead."

Nothing compares
No worries or cares
Regrets and mistakes
They are memories made.
Who would have known how bittersweet this would taste?

Never mind, I'll find someone like you
I wish nothing but the best for you too
Don't forget me, I beg
I'll remember you said,
"Sometimes it lasts in love but sometimes it hurts instead,
Sometimes it lasts in love but sometimes it hurts instead."""

**Dolly Parton - Jolene** 

"""Jolene, Jolene, Jolene, Jolene
I'm begging of you please don't take my man
Jolene, Jolene, Jolene, Jolene
Please don't take him just because you can

Your beauty is beyond compare
With flaming locks of auburn hair
With ivory skin and eyes of emerald green

Your smile is like a breath of spring
Your voice is soft like summer rain
And I cannot compete with you, Jolene

He talks about you in his sleep
There's nothing I can do to keep
From crying when he calls your name, Jolene

And I can easily understand
How you could easily take my man
But you don't know what he means to me, Jolene

Jolene, Jolene, Jolene, Jolene
I'm begging of you please don't take my man
Jolene, Jolene, Jolene, Jolene
Please don't take him just because you can

You could have your choice of men
But I could never love again
He's the only one for me, Jolene

I had to have this talk with you
My happiness depends on you
And whatever you decide to do, Jolene

Jolene, Jolene, Jolene, Jolene
I'm begging of you please don't take my man
Jolene, Jolene, Jolene, Jolene
Please don't take him even though you can

Jolene, Jolene"""


**Louis Armstrong - What a wonderful world**

"""I see trees of green
Red roses too
I see them bloom
For me and you
And I think to myself
What a wonderful world

I see skies of blue
And clouds of white
The bright blessed day
The dark sacred night
And I think to myself
What a wonderful world

The colors of the rainbow
So pretty in the sky
Are also on the faces
Of people going by
I see friends shaking hands
Saying, "How do you do?"
They're really saying
"I love you"

I hear babies cry
I watch them grow
They'll learn much more
Than I'll never know
And I think to myself
What a wonderful world

Yes, I think to myself
What a wonderful world

Oh yeah"""




**Rage against the Machine - Killing in the Name**


"""
Killing in the name of
Some of those that work forces, are the same that burn crosses
Some of those that work forces, are the same that burn crosses
Some of those that work forces, are the same that burn crosses
Some of those that work forces, are the same that burn crosses
Huh!

Killing in the name of
Killing in the name of

And now you do what they told ya
And now you do what they told ya
And now you do what they told ya
And now you do what they told ya
And now you do what they told ya
And now you do what they told ya
And now you do what they told ya
But now you do what they told ya
Well now you do what they told ya

Those who died are justified, for wearing the badge, they're the chosen whites
You justify those that died by wearing the badge, they're the chosen whites
Those who died are justified, for wearing the badge, they're the chosen whites
You justify those that died by wearing the badge, they're the chosen whites

Some of those that work forces, are the same that burn crosses
Some of those that work forces, are the same that burn crosses
Some of those that work forces, are the same that burn crosses
Some of those that work forces, are the same that burn crosses
Uggh!

Killing in the name of
Killing in the name of


Those who died are justified, for wearing the badge, they're the chosen whites
You justify those that died by wearing the badge, they're the chosen whites
Those who died are justified, for wearing the badge, they're the chosen whites
You justify those that died by wearing the badge, they're the chosen whites
Come on!

Yeah! Come on!

Fuck you, I won't do what you tell me
Fuck you, I won't do what you tell me
Fuck you, I won't do what you tell me
Fuck you, I won't do what you tell me
Fuck you, I won't do what you tell me
Fuck you, I won't do what you tell me
Fuck you, I won't do what you tell me
Fuck you, I won't do what you tell me

Motherfucker!
Uggh!
"""


**Three Days Grace - I hate everything about you**


"""Every time we lie awake
After every hit we take
Every feeling that I get
But I haven't missed you yet

Every roommate kept awake
By every sigh and scream we make
All the feelings that I get
But I still don't miss you yet

Only when I stop to think about it

I hate everything about you
Why do I love you?
I hate everything about you
Why do I love you?

Every time we lie awake
After every hit we take
Every feeling that I get
But I haven't missed you yet

Only when I stop to think about it

I hate everything about you
Why do I love you?
I hate everything about you
Why do I love you?

Only when I stop to think about you
I know
Only when you stop to think about me
Do you know?

I hate everything about you
Why do I love you?
You hate everything about me
Why do you love me?

I hate
You hate
I hate
You love me

I hate everything about you
Why do I love you?"""